# Segmentation

## Importing dependencies

In [ ]:
import numpy as np
from keras.models import load_model
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
import openslide as op
from PIL import Image
from scipy import misc
import os
import time
import h5py
import configparser
import ast
from util import mkdirs, extend_glob, file_suffix, remove_small_regions

## Configuration

In [ ]:
config = configparser.RawConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read('cytonet.cfg')
section = 'segmentation'

In [ ]:
# Paths
filenames = ast.literal_eval(config.get(section, 'filenames'))
mask_pattern = config.get(section, 'mask_pattern')
matrice_file = h5py.File(config.get(section, 'matrice_file') if config.has_option(section, 'matrice_file') \
                         else config.get('saving', 'output_file'),'r')
output_pattern= config.get(section, 'output_pattern')

experiment_folder = config.get(section, 'experiment_folder') if config.has_option(section, 'experiment_folder') \
                    else config.get('general', 'experiment_folder')
model_name = os.path.join(experiment_folder,  "matrices/" , 'model.099.hdf5')

load_level=config.getint(section, 'load_level') if config.has_option(section, 'load_level') else config.getint('general', 'load_level')
patch_size = config.getint(section, 'patch_size') if config.has_option(section, 'patch_size') else config.getint('general', 'patch_size')
stride = eval(config.get(section, 'stride'))

color_channels = config.getint(section, 'color_channels') if config.has_option(section, 'color_channels') \
                else config.getint('general', 'color_channels')

patch_shape = (patch_size, patch_size,color_channels) # Shape of the image (patch)
patch_mask_shape = (patch_size, patch_size)   # Shape of the mask (patch)
threshold = config.getfloat(section, 'threshold') 

## Image loading

In [ ]:
files = extend_glob(filenames)

In [ ]:
#Need to find a proper way to do this

from keras import backend as K
from keras.backend.common import _EPSILON
import tensorflow as tf

def _to_tensor(x, dtype):
    """Convert the input `x` to a tensor of type `dtype`.
    # Arguments
        x: An object to be converted (numpy array, list, tensors).
        dtype: The destination type.
    # Returns
        A tensor.
    """
    x = tf.convert_to_tensor(x)
    if x.dtype != dtype:
        x = tf.cast(x, dtype)
    return x

def binary_crossentropy2K(output, target, from_logits=False):
    """Binary crossentropy between an output tensor and a target tensor.
    # Arguments
        output: A tensor.
        target: A tensor with the same shape as `output`.
        from_logits: Whether `output` is expected to be a logits tensor.
            By default, we consider that `output`
            encodes a probability distribution.
    # Returns
        A tensor.
    """
    # Note: tf.nn.sigmoid_cross_entropy_with_logits
    # expects logits, Keras expects probabilities.
    if not from_logits:
        # transform back to logits
        epsilon = _to_tensor(_EPSILON, output.dtype.base_dtype)
        output = tf.clip_by_value(output, epsilon, 1 - epsilon)
        output = tf.log(output / (1 - output))

    return tf.nn.sigmoid_cross_entropy_with_logits(labels=target,
                                                   logits=output)

def binary_crossentropy2(y_true, y_pred):
    #print("test:")
    #print(binary_crossentropy2K(y_pred, y_true))
    #print(K.mean(binary_crossentropy2K(y_pred, y_true), axis=-1))
    #print("test2:")
    return binary_crossentropy2K(y_pred, y_true)

In [ ]:
import keras.losses
keras.losses.binary_crossentropy2 = binary_crossentropy2
#Another way to load the custom loss
#model = load_model('model/multi_task/try.h5', custom_objects={'loss_max': loss_max})

# Load model
UNet = load_model(model_name)

### Loading base picture

In [ ]:
for file in files:
    output_folder = os.path.join(experiment_folder, "prediction", os.path.splitext(os.path.basename(file))[0])
    output_folder_jet = os.path.join(output_folder,"proba", "jet")
    output_folder_gray = os.path.join(output_folder,"proba", "grayscale")
    mkdirs(output_folder, 0o777)
    mkdirs(output_folder_jet, 0o777)
    mkdirs(output_folder_gray, 0o777)
    
    maskname = file_suffix(file, mask_pattern)
    im = op.OpenSlide(file)
    imload = np.asfarray(im.read_region((0,0), load_level, im.level_dimensions[load_level]),dtype=np.float32)[:,:,0:color_channels]/255
    
    if use_training_norm:
        stats = matrice_file['stats'][:]
        im_mean=stats[0]
        im_std=stats[1]
        matrice_file.close()

        imload-=im_mean
        imload/=im_std
    else:
        imload-=imload.mean()
        imload/=imload.std()

    maskload = Image.open(maskname)
    maskload = np.expand_dims(np.array(maskload.resize(im.level_dimensions[load_level]),dtype=np.float32),-1)

    outputProba = np.zeros((im.level_dimensions[load_level][1],im.level_dimensions[load_level][0],nb_classes),dtype=np.float32)
    outputProbaTimes = np.zeros((im.level_dimensions[load_level][1],im.level_dimensions[load_level][0]),dtype=np.uint8)

    # Prediction
    for pos_x in range(0,im.level_dimensions[level][0],stride):
        offset_x = pos_x+patch_size
        for pos_y in range(0,im.level_dimensions[level][1],stride):
            offset_y = pos_y+patch_size


            if im.level_dimensions[level][1] < offset_y:
                pos_y = im.level_dimensions[level][1]-patch_size
                offset_y = pos_y+patch_size


            if im.level_dimensions[level][0] < offset_x:
                pos_x = im.level_dimensions[level][0]-patch_size
                offset_x = pos_x+patch_size

            xx = np.expand_dims(imload[pos_y:offset_y ,pos_x:offset_x,:],0)
            yy = maskload[pos_y:offset_y,pos_x:offset_x,:]

            #pred = UNet.predict(xx)[..., 1].reshape(patchShape[:2])
            pred = UNet.predict(xx)[0,:,:,:]

            part=output_proba[pos_y:offset_y,pos_x:offset_x]       # Creating a pointer on the area we just predicted
            part[:,:]+=pred[0:part.shape[0],0:part.shape[1]]       # Filling the pointer with the information gathered from the prediction
            output_proba_times[pos_y:offset_y,pos_x:offset_x]+=1    # Counting the amount of patches applying on every single pixel 

    t = time.clock()
    print("Execution duration : ", t-t0)

    #TODO: automate this, depending on the number of classes
    
    #Dividing pixels' values by the number of times the patch passed on it 
    output_proba[:,:,0]/=output_proba_times
    output_proba[:,:,1]/=output_proba_times
    output_proba[:,:,2]/=output_proba_times
    
    
    output_proba1 = output_proba[:,:,0]
    output_proba2 = output_proba[:,:,1]
    output_proba3 = output_proba[:,:,2]
    
    if show_prediction:
        plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
        plt.subplot(211)
        plt.imshow(output_proba1,interpolation='none', cmap="jet")
        plt.show()

        plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
        plt.subplot(211)
        plt.imshow(output_proba2,interpolation='none', cmap="jet")
        plt.show()

        plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
        plt.subplot(211)
        plt.imshow(output_proba3,interpolation='none', cmap="jet")
        plt.show()
    
    #Saving probabilities
    misc.imsave(os.path.join(output_folder_gray, "proba1_gray.png"), output_proba1)
    misc.imsave(os.path.join(output_folder_gray, "proba2_gray.png"), output_proba2)
    misc.imsave(os.path.join(output_folder_gray, "proba3_gray.png"), output_proba3)
    
    plt.imsave(os.path.join(output_folder_jet, "proba1.png"), output_proba1, cmap=plt.cm.jet)
    plt.imsave(os.path.join(output_folder_jet, "proba2.png"), output_proba2, cmap=plt.cm.jet)
    plt.imsave(os.path.join(output_folder_jet, "proba3.png"), output_proba3, cmap=plt.cm.jet)